# 1. Preparamos Docker


**Windows 10/11:**  
- Instala **Docker Desktop** (activar WSL 2/Hyper-V si es necesario).  
- Reinicia y comprueba que Docker esté activo.

**Probar Docker**
```bash
docker --version
docker run --rm hello-world
```

**Levantar los contenedores**
```bash
docker compose up -d
```

**Verificar**
```bash
docker compose ps
```



**Probar las UIs**

-phpMyAdmin: (MYSQL)

http://localhost:8081


Server: mysql · User: root · Pass: etl25 

→ debe verse workshop1_raw_db.



-pgAdmin:    (POSTGRESQL)

http://localhost:8080


Login: etl@uao.com / etl25 

→ Add Server: Host postgres-dw, DB workshop1_transformed_db, User transformed, Pass etl25.





# 2. Instalar UV (gestor de entornos y paquetes)

**Linux/macOS:**
```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
source ~/.bashrc  # o ~/.zshrc
uv --version
```
**Windows (PowerShell):**
```powershell
irm https://astral.sh/uv/install.ps1 | iex
uv --version
```


**Preparar ambiente con UV (desde tu terminal)**


```bash

# Activar entorno virtual

# Linux/macOS:
uv venv .venv
source .venv/bin/activate
# Windows (PowerShell):
uv venv .venv
uv sync
.\.venv\Scripts\Activate.ps1

uv pip install ipykernel jupyter
python -m ipykernel install --user --name workshop1 --display-name "Python (workshop1)"

```



# 3. Creamos la base de datos (MYSQL) workshop1_raw_db  

la variable de entorno MYSQL_DATABASE ya la creó automáticamente al iniciar el contenedor. 

Si no fuera así:

```bash
docker exec -it workshop1-mysqlDB mysql -uroot -p
-- Password: etl25
CREATE DATABASE IF NOT EXISTS workshop1_raw_db;
SHOW DATABASES;
EXIT;

```

# 4. Creamos la tabla raw_candidates en la base de datos de (MYSQL) workshop1_raw_db  

In [4]:
from sqlalchemy import create_engine, text
import pandas as pd

In [5]:
df = pd.read_csv("../data/candidates.csv") 
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 1 columns):
 #   Column                                                                                                                       Non-Null Count  Dtype 
---  ------                                                                                                                       --------------  ----- 
 0   First Name;Last Name;Email;Application Date;Country;YOE;Seniority;Technology;Code Challenge Score;Technical Interview Score  50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


In [ ]:
"""
usuario     = "root"
contraseña  = "etl25"
host        = "localhost"
puerto      = "3306"
base_datos  = "workshop1_raw_db"

url = f"mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
engine = create_engine(url)

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS raw_candidates"))
    conn.commit()
    print("Tabla raw_candidates eliminada")
"""

🗑️ Tabla raw_candidates eliminada


In [9]:
# 1. Conexión a MySQL con SQLAlchemy 

usuario     = "root"
contraseña  = "etl25"
host        = "localhost"
puerto      = "3306"
base_datos  = "workshop1_raw_db"

# 2. Crear conexión SQLAlchemy (driver: mysql-connector)
url = f"mysql+mysqlconnector://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}"
engine = create_engine(url)

# 3. Crear tabla con PRIMARY KEY
with engine.connect() as conn:
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS raw_candidates (
        id INT AUTO_INCREMENT PRIMARY KEY,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        email VARCHAR(150),
        application_date DATE,
        country VARCHAR(60),
        years_of_experience INT,
        seniority VARCHAR(50),
        technology VARCHAR(100),
        code_challenge_score FLOAT,
        technical_interview_score FLOAT
    )
    """))
    conn.commit()

# 4. Leer CSV y renombrar columnas
df = pd.read_csv("../data/candidates.csv", sep=";")
df = df.rename(columns={
    "First Name":"first_name",
    "Last Name":"last_name",
    "Email":"email",
    "Application Date":"application_date",
    "Country":"country",
    "YOE":"years_of_experience",
    "Seniority":"seniority",
    "Technology":"technology",
    "Code Challenge Score":"code_challenge_score",
    "Technical Interview Score":"technical_interview_score",
})

# 5. Insertar en la tabla (MySQL manejará el id automáticamente)
df.to_sql("raw_candidates", con=engine, if_exists="append", index=False)
print("✅ Datos insertados:", len(df))



✅ Datos insertados: 50000
